# Piles

On redonne l’interface des piles en `OCaml`. On utilisera directement
ces fonctions dans la suite.

In [73]:
let creer_pile () = ref []
let empile p x = p := x :: !p
let depile p = match !p with
    | [] -> failwith "pile vide"
    | t::q -> p := q; t
let est_vide p = !p = []

val creer_pile : unit -> 'a list ref = <fun>
val empile : 'a list ref -> 'a -> unit = <fun>
val depile : 'a list ref -> 'a = <fun>
val est_vide : 'a list ref -> bool = <fun>

Comme on va être amenés à depiler en ignorant la valeur renvoyée,
`OCaml` va faire un warning car on jette la valeur alors qu’elle n’est
pas de type `unit`. Pour éviter ce warning, on utilise la fonction
`ignore` : `ignore (depile p)` pourra ainsi être appelé pour retirer une
valeur de la pile en l’ignorant.

## Expressions bien parenthésées

### Avec un seul type de parenthèses

On va considérer des chaînes de caractères contenant des parenthèses et
des lettres comme `"(a(bc)a)"` et on va chercher à savoir si elles sont
bien parenthésées : c’est-à-dire que toutes les parenthèses ouvertes
sont bien fermées. Pour cela, il suffit de compter le nombre qui sont
ouvertes et de s’assurer à la fin que ce nombre vaut 0.

Rappel, on accède au ième caractère de la chaine `s` avec `s.[i]` et on
obtient sa taille avec `String.length`.


In [16]:
let bien_parenthesee s : bool =
    let acc' c acc =
        match c with 
            '(' -> (acc+1) 
            | ')' -> (acc-1)
            | _ -> acc
    in
    
    let rec aux i acc =
        if i = String.length s then acc = 0 else
        if acc<0 then false else
        aux (i+1) (acc' s.[i] acc)
    in aux 0 0

val bien_parenthesee : string -> bool = <fun>

In [17]:
assert(bien_parenthesee "(a(bc)a)");
assert(not (bien_parenthesee "(a(bc)a"));
assert(not (bien_parenthesee "(a)bc)a("));
assert(bien_parenthesee "(()(()))((())())");
"OK"

- : string = "OK"



### Avec plusieurs types de parenthèses

Maintenant, on va considérer une liste l de couples `(ouvrant,fermant)`
comme `('(', ')')` ou `('{','}')` montrant quelles les types de
parenthèses autorisées et on va poser la même question du parenthésage.
Ainsi avec la liste `let l = [('(', ')'); ('{', '}'); ('[', ']') ]` la
chaine `(a{bc})[a(b)]` est bien parenthésées mais `'(a}'` ne l’est pas
(pas le même symbole qui ferme) et `([(b)]` non plus (pas de `')'`
fermante).

On ne va donc plus pouvoir utiliser un simple entier. On ne peut pas non
plus utiliser un entier par symbole car ça ne permettra pas de détecter
des problèmes comme `([)]` (*exercice : vous en convaincre*). On va donc
utiliser une pile de symbole de fermeture attendu. Au départ, la pile
est vide, ensuite, à chaque fois qu’on voit un symbole d’ouverture on
empile le symbole de fermeture correspondant, et si on voit un symbole
de fermeture on s’assure que ce soit le sommet de la pile.


In [4]:
let bien_parenthesee_l (l:(char * char) list) (s:string) : bool =
    let is_opening c =
        let rec aux_open l =
            match l with
            []->false
            |(t,_)::q ->  if t=c then true else aux_open q 
        in aux_open l
    in
    
    let closing_of c =
        let rec aux_close l =
            match l with
            []-> failwith "can't close damnit"
            |(t,t')::q ->  if t=c then t' else aux_close q 
        in aux_close l
    in
    
    let suiv c acc =
        if is_opening c then c::acc
        else 
        if acc = [] then acc 
        else 
        if c = closing_of (List.hd acc) then List.tl acc
        else acc
    in
    
    let rec aux i pile =
        if i = String.length s then pile = [] else
        aux (i+1) (suiv s.[i] pile)
    in aux 0 []

val bien_parenthesee_l : (char * char) list -> string -> bool = <fun>

In [8]:
assert(bien_parenthesee_l [('(',')')] "(a(bc)a)");
assert(not (bien_parenthesee_l [('(',')')] "(a(bc)a"));
assert(not (bien_parenthesee_l [('(',')')] "(a)bc)a("));
assert(bien_parenthesee_l [('(',')')] "(()(()))((())())");
assert(bien_parenthesee_l [('(', ')'); ('{', '}'); ('[', ']')] "([]{}){[()]()}");
assert(not (bien_parenthesee_l [('(', ')'); ('{', '}'); ('[', ']')] "([)]"));
assert(not (bien_parenthesee_l [('(', ')'); ('{', '}'); ('[', ']')] "([)}"));
"Sartek bg"

- : string = "Sartek bg"


## Pile pour évaluer des expressions

### Forme postfixe

On va considérer ici des expressions écrites sous forme postfixe.

La présentation traditionnelle des expressions est infixe : le symbole
d’opération est au milieu des opérandes, comme dans `2 + 3`. Cette
notation est naturelle mais a le désavantage de nécessiter des
parenthèses comme dans `(2 + 3) * 5`. Une autre possibilité est d’avoir
une notation postfixe `2 3 +` où le symbole vient après ses opérandes.
L’avantage est de permettre de ne pas avoir besoin de parenthèse, en
effet, on peut écrire `2 3 + 5 *` dans l’exemple précédent.

En `OCaml`, on représentera une telle expression sous la forme d’une
liste contenant :

-   soit des entier pour les constantes
-   soit un caractère parmi `'+', '-', '*', '/'` pour les opérations

Pour cela on définit le type suivant :

In [10]:
type op = Plus | Minus | Mult | Div
type token = C of int | O of op

type op = Plus | Minus | Mult | Div
type token = C of int | O of op

L’expression précédente correspond alors à la liste
`[ C 2; C 3; O Plus; C 5; O Mult ]`.

Pour évaluer une telle expression, on utilise une pile, initialement
vide, et on parcourt de gauche à droite l’expression postfixe. Si la
valeur lue est

-   un entier : on le place sur la pile
-   un symbole d’opération : on dépile deux valeurs sur la pile et on
    effectue l’opération, puis on empile le résultat.

Quand on a fini le parcours, il ne doit il y avoir qu’une seule valeur
sur la pile qui est le résultat (sinon, c’est un problème similaire à un
mauvais parenthèsage). On dit qu’une expression qui permet d’aboutir à
une unique valeur à la fin du parcours est **bien formée**.

Pour les opérations non commutatives comme `'-'` et `'/'`, on adopte la
convention que `a b -` calcule `a - b`. Il faudra donc faire attention à
respecter cela dans l’évaluation. **On effectue aussi des divisions
entières ici et non flottantes.**

On vous demande d’écrire une fonction d’évaluation des expressions.


In [39]:
let evalue (e : token list) : int =
    let rec aux l pile =
        match l with
        [] -> depile pile
        |t::q -> aux q (
        match t with
        | C i -> empile pile i; pile
        | O op -> 
            let a,b = (depile pile, depile pile) in
            empile pile (match op with
            | Plus ->  (a+b)
            | Minus -> (a-b) 
            | Mult ->  (a*b) 
            | Div ->  (a/b)
            );pile
        )
     in aux e (creer_pile ())
    

val evalue : token list -> int = <fun>

In [40]:
assert(evalue [C 1; C 2; O Plus] = 3);
assert(evalue [C 1; C 2; O Minus] = -1);
assert(evalue [C 3; C 1; O Minus; C 5; O Mult] = 10);
assert(evalue [C 1; C 2; O Plus;  C 2; O Div] = 1);
assert(evalue [C 1; C 2; C 3; C 4; C 5; C 6; O Plus; O Mult; O Div; O Minus; O Plus] = 3);
"OK"

- : string = "OK"

### Forme infixe - Algorithme de Dijkstra

On a vu comment évaluer une expression postfixe avec une pile, Dijkstra
a trouvé un algorithme utilisant deux piles pour évaluer une expression
infixe, c’est-à-dire une expression standard avec des parenthèses
$(1+(2\times 3)-(3\times (4 \times 2)))$.

**Attention** Il y a même des parenthèses autour de la première
expression et on place également les parenthèses pourtant inutiles des
opérations associatives. On verra plus tard qu’il s’agit ici d’un arbre
binaire.

On considère deux piles, une pile de nombres et une pile d’opérateurs.

Pour chaque symbole :

-   Si c’est une parenthèse ouvrante, on l’ignore ;
-   Si c’est un nombre on le pousse sur la pile de nombre ;
-   Si c’est un opérateur on le pousse sur la pile d’opérateurs ;
-   Si c’est une parenthèse fermante, on dépile un opérateur et autant
    de nombre que son arité (son nombre d’opérandes), puis on empile le
    résultat.

Pour simplifier, on ne considère ici que des opérations binaires. Il
faudra donc retirer deux nombres de la pile à chaque fois.

**Question**

Évaluer ainsi $(1+((2+3)\times (4\times 5)))$.

Pour simplifier, on ne va considérer que des chiffres, ainsi, il suffira
de s’assurer qu’un caractère est entre `'0'` et `'9'` pour le convertir
avec `int_of_char c - int_of_char '0'` en un `int`.

<div class="ui message orange"><div class="header">Remarque</div>

En `OCaml`, on peut filtrer une plage de caractères comme ici en
utilisant le filtrage `| '0'..'9' ->`

On vous demande de réaliser une fonction d'évaluation selon cette méthode en prenant une chaîne en entrée.

In [78]:
let evalue (s:string) : int =
    let faire op b a : int = match op with
                '+' -> a+b |'-' -> a-b |'*' -> a*b |'/' -> a/b | _ -> failwith "opérant very sus" in
    let rec aux i nbpile oppile =
        if i = String.length s then depile nbpile else
        begin
            begin
                match s.[i] with
                    | '+' |'-'|'*'|'/' -> empile oppile s.[i]
                    | '0'..'9' as c -> empile nbpile  (int_of_char c - int_of_char '0')
                    | ')' -> empile nbpile (faire (depile oppile) (depile nbpile) (depile nbpile))
                    |_-> ()
            end; aux (i+1) nbpile oppile
        end
        
    in aux 0 (creer_pile ()) (creer_pile ())

val evalue : string -> int = <fun>

In [67]:
assert (evalue "(1+2)" = 3);
assert (evalue "(2-1)" = 1);
assert (evalue "(3*(1+2))" = 9);
assert(evalue "(1+((2+3)*(4*5)))" = 101);
"OK"


- : string = "OK"


# File avec deux piles

On va implémenter une file avec deux piles.

On a vu qu’il suffisait de faire une `'a list ref` pour avoir une pile
en `OCaml`, mais pour réaliser une file, comme on va changer les deux
listes, on considère plutôt le type suivant :

In [19]:
type 'a file = { 
    mutable entree : 'a list;
    mutable sortie : 'a list
}

type 'a file = { mutable entree : 'a list; mutable sortie : 'a list; }

Ainsi, chaque champ `mutable` sera une sorte de référence.


In [20]:
let cree_file () : 'a file =
    {
        entree = [];
        sortie = []
    }


val cree_file : unit -> 'a file = <fun>

In [21]:
let est_vide (f:'a file) : bool =
    f.entree = [] && f.sortie = []


val est_vide : 'a file -> bool = <fun>

In [22]:
let enfile (f:'a file) (x:'a) : unit =
    f.entree <- x::f.entree


val enfile : 'a file -> 'a -> unit = <fun>

In [23]:
let transvase (f:'a file) : unit =
    assert(f.sortie = []);
    f.sortie <- List.rev f.entree;
    f.entree <- []


val transvase : 'a file -> unit = <fun>

In [24]:
let defile (f : 'a file) : 'a =
    if f.sortie = []
    then transvase f;
    match f.sortie with
    | [] -> failwith "File vide"
    | t::q -> f.sortie <- q; t


val defile : 'a file -> 'a = <fun>

In [25]:
let f = cree_file () in
Printf.printf "Test enfile: ";
assert(est_vide f);
enfile f 1;
enfile f 2;
enfile f 3;
Printf.printf "OK\n";
Printf.printf "Test transvase: ";
assert(f.entree = [3;2;1] && f.sortie = []);
transvase f;
Printf.printf "OK\n";
Printf.printf "Test defile: ";
assert(f.entree = [] && f.sortie = [1;2;3]);
assert(defile f = 1);
assert(defile f = 2);
enfile f 4;
assert(defile f = 3);
assert(defile f = 4);
assert(est_vide f);
Printf.printf "OK\n";


Test enfile: OK
Test transvase: OK
Test defile: OK


- : unit = ()